### Generate Stimuli

- Target should be in the center of the object
- Targets should be in the center of the image (x)
- Targets need to be a certain distance from the bottom and top of image
- targets should not be on the floor or ceiling

### Filter by Ground Plane 

If there is NO floor polygon in the object segmentation, then remove the image folder

In [377]:
import pandas as pd 
import json
from PIL import Image, ImageDraw

def load2DSeg(folderpath):
    return json.load(open(folderpath + '/annotation2Dfinal/index.json'))

    
def checkFloor(folder):
    """
    Args:
        folder - path to image folder 
    Returns:
        True if there is an object segmented labeled 'floor' in annotation2Dfinal
        False if there is no object segmented labeled 'floor' in annotation2Dfinal
    """
    try:
        data = load2DSeg(folder)
    except:
        print('loading json failed at ' + folder)
        return 
    
    imageDBNumber = data['fileList'][0][-11:-4]
    img = Image.open(folder + '/image/' + imageDBNumber + '.jpg').convert('RGBA')
    
    has_floor = False
    for poly in data['frames'][0]['polygon']:
        object_index = poly["object"]
        try:
            if data['objects'][object_index]['name'].lower() == 'floor':
                has_floor = True 
        except:
                return has_floor

    return has_floor

def main_checkFloor(rootdir):
    """
    Loop through directory of image folders and remove image folders that have NO floor object
    """
    count = 0 # number of images kept
    for _dir in os.listdir(rootdir):
        path = rootdir + '/' + _dir
        has_floor = checkFloor(path)
        if has_floor == True:
            count += 1
        else:
            if '.DS_Store' not in path:
                shutil.rmtree(path)
    
    return count
        

In [351]:
kinect2_subset = '/Users/prachi/Documents/depth_duration/kinect2data_subset'
main_checkFloor(kinect2_subset)

# some jsons will fail to load due to a parse error --> no cause for concern, these images will be skipped

loading json failed at /Users/prachi/Documents/depth_duration/kinect2data_subset/.DS_Store
loading json failed at /Users/prachi/Documents/depth_duration/kinect2data_subset/000504_2014-06-08_23-03-02_260595134347_rgbf000064-resize
loading json failed at /Users/prachi/Documents/depth_duration/kinect2data_subset/000631_2014-06-08_16-30-36_260595134347_rgbf000088-resize
loading json failed at /Users/prachi/Documents/depth_duration/kinect2data_subset/000667_2014-06-09_21-06-12_260595134347_rgbf000145-resize
loading json failed at /Users/prachi/Documents/depth_duration/kinect2data_subset/000668_2014-06-09_21-07-12_260595134347_rgbf000161-resize
loading json failed at /Users/prachi/Documents/depth_duration/kinect2data_subset/001177_2014-06-17_15-46-38_260595134347_rgbf000056-resize
loading json failed at /Users/prachi/Documents/depth_duration/kinect2data_subset/002295_2014-06-28_19-00-10_260595134347_rgbf000138-resize
loading json failed at /Users/prachi/Documents/depth_duration/kinect2data_s

3085

### Filter Stimuli Folder by Extrinsics

In [353]:
from scipy.spatial.transform import Rotation as R
import math
import numpy as np 

#converts the rotation matrix in extrinsics.txt from string to a numpy array 
def rotationmatrix(matrix_path):
    matrixtext = open(matrix_path).read()
    
    x00 = float(matrixtext.split('\t')[0].split(' ')[0])
    x01 = float(matrixtext.split('\n')[0].split(' ')[1])
    x02 = float(matrixtext.split('\n')[0].split(' ')[2])
    
    x10 = float(matrixtext.split('\n')[1].split(' ')[0])
    x11 = float(matrixtext.split('\n')[1].split(' ')[1])
    x12 = float(matrixtext.split('\n')[1].split(' ')[2])
    
    x20 = float(matrixtext.split('\n')[2].split(' ')[0])
    x21 = float(matrixtext.split('\n')[2].split(' ')[1])
    x22 = float(matrixtext.split('\n')[2].split(' ')[2])
    
    #list of the matrix values excluding the 3rd column of just zeroes 
    matrixlist = [[x00, x01, x02], [x10, x11, x12], [x20, x21, x22]]
    
    #creates 3x3 array of the rotation matrix
    matrixarray = np.full((3,3), matrixlist)
    
    return matrixarray

def scipy_extrinsics(extrinsics_path):

    matrix = rotationmatrix(extrinsics_path)

    r = R.from_matrix(matrix)
    r = r.as_euler('zyx', degrees=True)

    # Roll Pitch Yaw XYZ (g,b,a) ¤ Euler ZYX (a,b,g) 

    rpy = {'roll': r[2], 'pitch': r[1] , 'yaw': r[0]}
    return rpy

def filter_by_extrinsics(rootdir, range_yaw, range_pitch, range_roll):
    """
    Args:
        rootdir: kinect2data folder path 
    Removes image folders that have extrinsics values outside of the set range 
    """
    count = 0 # number of images kept
    excluded = 0
    for _dir in os.listdir(rootdir):
        path = rootdir + '/' + _dir
        if _dir != '.DS_Store':
            extpath = path + '/extrinsics'
            os.listdir(extpath)
            fname = os.listdir(extpath)[0]
            if fname != '.ipynb_checkpoints':
                try:
                    extrinsics = scipy_extrinsics(extpath + '/' + fname)
                    roll = extrinsics['roll']
                    pitch = extrinsics['pitch']
                    yaw = extrinsics['yaw']
                except:
                    print( path, None)
                if range_yaw[0] <= yaw <= range_yaw[1]:
                    count += 0
                else:
                    try:
                        shutil.rmtree(path)
                    except:
                        pass
                if range_pitch[0] <= pitch <= range_pitch[1]:
                    count += 0
                else:
                    try:
                        shutil.rmtree(path)
                    except:
                        pass      
                if range_roll[0] <= roll <= range_roll[1]:
                    count += 1
                else:
                    try:
                        shutil.rmtree(path)
                    except:
                        pass

                
                
    
    return count, excluded 
                    
    
    

In [430]:
kinect2_subset = '/Users/prachi/Documents/depth_duration/kinect2data_subset'
yaw_range = [-0.5, 0.5]
pitch_range = [-0.5,0.5]
roll_range = [-30, -10]
incl, excl = filter_by_extrinsics(kinect2_subset, yaw_range, pitch_range, roll_range)
print(incl, excl)

2599 0


#### Count the number of image folder left after extrinsics and ground plane cleaning

In [438]:
import os

imagefolders = 0

for filenames in os.listdir(kinect2_subset):
    imagefolders += 1

print("Total image folders left: ", imagefolders)

Total image folders left:  721


### Create stimuli using subset of kinect2 

In [7]:
from PIL import Image

def getAbsoluteDepth(filepath, x, y):
    depth = getPixelValue(filepath, x, y)
    return depth / 8000

def getPixelValue(filepath, x, y):
    im = Image.open(filepath)
    px = im.load()
    return px[x, y]


In [8]:
# from PIL import Image, ImageDraw
import json
from pathlib import Path
from shapely.geometry import Polygon, LineString, Point
import shapely.errors as sherrors


def allCenterObjects(folder, exit):
    uniqueID = folder[-57:-51]
    data = json.load(open(folder + '/annotation2Dfinal/index.json'))
    imageDBNumber = data['fileList'][0][-11:-4]
    output = {
        'dbPath': folder,
        'imagePath': folder + '/image/' + imageDBNumber + '.jpg',
        'objects': []
    }
    # img = Image.open(output['imagePath']).copy()
    depthpath = folder + '/depth_bfx/' + imageDBNumber + '.png'
    for poly in data['frames'][0]['polygon']:
        if min(poly['x']) < 364 and max(poly['x']) > 364:
            x = poly['x']
            y = poly['y']
            zipped = list(zip(x, y))
            a = Polygon(zipped)
            # 364 is the horizontal midpoint 
            # full vertical range is acceptable
            b = LineString([(364, 0), (364, 529)])
            crossingpoints = []
            try:
                coords = a.intersection(b).coords
            except NotImplementedError:
                coords = []
                for line in list(a.intersection(b)):
                    for coord in line.coords:
                        coords.append(coord)
            except sherrors.TopologicalError as toperr:
                print(toperr, folder, data['objects'][poly['object']]['name'])
                return {'path': folder.split('Desktop/')[1], 'type': 'topology'}
            for intersection in coords:
                # draw = ImageDraw.Draw(img)
                if intersection[1] < 0 or intersection[1] > 529:
                    return {'path': folder.split('Desktop/')[1], 'type': 'negative_cross'}
                if intersection[1] >= 5:
                    upy = intersection[1] - 5
                else:
                    upy = -1000
                if intersection[1] <= 524:
                    downy = intersection[1] + 5
                else:
                    downy = 3000
                up = Point(intersection[0], upy)
                down = Point(intersection[0], downy)
                if a.contains(up):
                    # draw.polygon([(364, upy), (364, upy)], 'red')
                    crossingpoints.append({'x': 364, 'y': upy, 'depth': getAbsoluteDepth(depthpath, 364, upy)})
                    # del draw
                elif a.contains(down):
                    # draw.polygon([(364, downy), (364, downy)], 'red')
                    crossingpoints.append({'x': 364, 'y': downy, 'depth': getAbsoluteDepth(depthpath, 364, downy)})
                    # del draw
                else:
                    crossingpoints.append({'x': intersection[0], 'y': intersection[1], 'depth': getAbsoluteDepth(depthpath, 364, intersection[1])})
            try:
                output['objects'].append({
                    'name': data['objects'][poly['object']]['name'],
                    'dbID': poly['object'],
                    'crossing_points': crossingpoints
                })
            except IndexError:
                print(folder, poly['object'])
                return {'path': folder.split('Desktop/')[1], 'type': 'object_index'}

    return output



In [435]:
exclude = ['000347_2014-06-09_19-49-03_260595134347_rgbf000225-resize', '003275_2014-05-14_16-10-31_094959634447_rgbf000100-resize',
                  '002120_2014-06-25_20-34-00_260595134347_rgbf000087-resize', '002486_2014-06-28_20-42-55_260595134347_rgbf000029-resize', 
                  '000354_2014-06-09_20-31-55_260595134347_rgbf000172-resize', '003351_2014-05-20_11-38-25_094959634447_rgbf000027-resize',
                  '002623_2014-06-24_14-44-20_094959634447_rgbf000038-resize', '001841_2014-06-22_13-18-35_260595134347_rgbf000048-resize',
                  '002475_2014-06-28_20-40-15_260595134347_rgbf000021-resize', '003474_2014-05-21_15-03-14_094959634447_rgbf002451-resize',
                  '003586_2014-05-23_16-47-01_094959634447_rgbf000100-resize', '002483_2014-06-28_20-42-10_260595134347_rgbf000029-resize',
                  '003154_2014-05-12_11-28-46_094959634447_rgbf000101-resize', '002808_2014-06-22_20-07-31_094959634447_rgbf000079-resize',
                  '002560_2014-06-24_13-48-11_094959634447_rgbf000075-resize', '000579_2014-06-08_11-07-06_260595134347_rgbf000166-resize', 
                  '002727_2014-06-22_11-49-24_094959634447_rgbf000025-resize', '002434_2014-06-28_20-30-03_260595134347_rgbf000054-resize',
                 '002093_2014-06-25_19-50-26_260595134347_rgbf000160-resize', 
                 '003044_2014-06-15_13-36-59_094959634447_rgbf000150-resize', '000903_2014-06-09_16-11-40_260595134347_rgbf000026-resize', 
                 '002091_2014-06-25_19-49-40_260595134347_rgbf000088-resize', 
                 '003025_2014-06-15_13-14-53_094959634447_rgbf000112-resize', '002291_2014-06-28_18-58-48_260595134347_rgbf000033-resize',
                 '003099_2014-06-24_21-04-53_094959634447_rgbf000076-resize', 
                 '002090_2014-06-25_19-49-03_260595134347_rgbf000064-resize', '001371_2014-06-19_15-53-51_260595134347_rgbf000049-resize',
                 '001931_2014-06-22_14-21-36_260595134347_rgbf000196-resize', '002443_2014-06-28_20-31-53_260595134347_rgbf000050-resize',
                 '000899_2014-06-09_15-49-21_260595134347_rgbf000086-resize', 
                 '000888_2014-06-09_15-43-40_260595134347_rgbf000119-resize',
                 '000689_2014-06-08_13-06-12_260595134347_rgbf000089-resize', '000877_2014-06-09_21-02-47_260595134347_rgbf000098-resize',
                 '002312_2014-06-28_19-58-29_260595134347_rgbf000084-resize', '000582_2014-06-09_16-27-17_260595134347_rgbf000224-resize',
                 '000954_2014-06-08_11-47-52_260595134347_rgbf000104-resize', '002142_2014-06-25_21-20-49_260595134347_rgbf000163-resize',
                 '002755_2014-06-22_19-10-02_094959634447_rgbf000076-resize', '001179_2014-06-17_15-47-33_260595134347_rgbf000093-resize',
                 '000924_2014-06-09_16-30-21_260595134347_rgbf000115-resize', 
                 '003510_2014-05-21_17-32-00_094959634447_rgbf000101-resize',
                 '002701_2014-06-22_11-27-34_094959634447_rgbf000075-resize',  
                 '002135_2014-06-25_21-17-43_260595134347_rgbf000087-resize', '000842_2014-06-08_17-05-51_260595134347_rgbf000104-resize',
                 '002382_2014-06-28_20-15-35_260595134347_rgbf000046-resize', '000442_2014-06-08_11-44-10_260595134347_rgbf000225-resize', 
                 '002133_2014-06-25_21-12-52_260595134347_rgbf000082-resize', '001067_2014-06-08_22-04-00_260595134347_rgbf000049-resize',
                 '003096_2014-06-24_21-03-34_094959634447_rgbf000070-resize', 
                 '002772_2014-06-22_19-22-49_094959634447_rgbf000059-resize','003229_2014-05-14_13-43-11_094959634447_rgbf000100-resize', '001165_2014-06-17_15-36-50_260595134347_rgbf000127-resize',
                 '003532_2014-05-22_15-05-18_094959634447_rgbf000101-resize', '002172_2014-06-24_21-39-57_260595134347_rgbf000101-resize',
                 '003058_2014-06-15_14-02-44_094959634447_rgbf000150-resize', '001948_2014-06-24_15-42-56_260595134347_rgbf000060-resize',
                 
                 '000459_2014-06-08_16-54-11_260595134347_rgbf000096-resize', '002929_2014-06-26_15-49-31_094959634447_rgbf000076-resize',
                 '000621_2014-06-08_11-21-23_260595134347_rgbf000147-resize', '001820_2014-06-26_20-54-15_260595134347_rgbf000032-resize',
                 '002471_2014-06-28_20-39-28_260595134347_rgbf000025-resize',
                 '002396_2014-06-28_20-20-31_260595134347_rgbf000072-resize',
                 '000694_2014-06-08_13-12-30_260595134347_rgbf000219-resize', '002339_2014-06-28_20-05-27_260595134347_rgbf000042-resize',
                 '003557_2014-05-23_10-25-16_094959634447_rgbf006128-resize', '000566_2014-06-09_22-50-50_260595134347_rgbf000080-resize',
                 '002402_2014-06-28_20-21-52_260595134347_rgbf000023-resize', '002794_2014-06-22_19-38-01_094959634447_rgbf000082-resize',
                 '002092_2014-06-25_19-49-59_260595134347_rgbf000128-resize', '002430_2014-06-28_20-29-09_260595134347_rgbf000047-resize',
                 '002547_2014-06-24_13-39-39_094959634447_rgbf000079-resize', '002651_2014-06-24_14-57-10_094959634447_rgbf000064-resize',
                 '002354_2014-06-28_20-08-27_260595134347_rgbf000049-resize', '003136_2014-05-12_10-58-05_094959634447_rgbf000100-resize',
                  '003187_2014-05-12_22-08-19_094959634447_rgbf000051-resize', '003041_2014-06-15_13-35-37_094959634447_rgbf000077-resize',
                  '002381_2014-06-28_20-15-26_260595134347_rgbf000027-resize', '001293_2014-06-16_16-27-24_260595134347_rgbf000103-resize',
                  '002724_2014-06-22_11-47-39_094959634447_rgbf000080-resize', '002520_2014-06-24_13-24-20_094959634447_rgbf000076-resize',
                  '001613_2014-06-20_11-32-48_260595134347_rgbf000054-resize', '001182_2014-06-17_15-51-16_260595134347_rgbf000070-resize',
                  '002601_2014-06-24_14-28-15_094959634447_rgbf000032-resize', '002436_2014-06-28_20-30-21_260595134347_rgbf000094-resize',
                  '003383_2014-05-20_14-41-21_094959634447_rgbf000101-resize', '002552_2014-06-24_13-46-11_094959634447_rgbf000074-resize',
                  '000790_2014-06-08_22-35-19_260595134347_rgbf000085-resize', '003409_2014-05-21_09-12-10_094959634447_rgbf000100-resize',
                  '002176_2014-06-25_19-50-50_260595134347_rgbf000073-resize', '000348_2014-06-09_19-49-30_260595134347_rgbf000130-resize',
                  '002331_2014-06-28_20-03-43_260595134347_rgbf000074-resize', '002834_2014-06-22_20-37-29_094959634447_rgbf000089-resize',
                  '003639_2014-05-23_22-09-41_094959634447_rgbf000055-resize', '002470_2014-06-28_20-39-13_260595134347_rgbf000029-resize',
                  '000487_2014-06-09_20-47-43_260595134347_rgbf000173-resize', '003227_2014-05-14_13-42-10_094959634447_rgbf000100-resize',
                  '002536_2014-06-24_13-36-30_094959634447_rgbf000088-resize', '003591_2014-05-23_16-57-22_094959634447_rgbf002866-resize',
                  '000002_2014-05-26_14-23-37_260595134347_rgbf000103-resize', '001513_2014-06-20_16-23-55_260595134347_rgbf000118-resize',
                  '001339_2014-06-16_17-04-45_260595134347_rgbf000065-resize', '003295_2014-05-14_22-20-22_094959634447_rgbf000100-resize',
                  '001430_2014-06-19_16-50-36_260595134347_rgbf000047-resize', '000370_2014-06-09_15-58-58_260595134347_rgbf000076-resize',
                  '001427_2014-06-19_16-49-12_260595134347_rgbf000035-resize', '002961_2014-06-08_17-40-53_094959634447_rgbf000144-resize',
                  '001435_2014-06-19_16-52-03_260595134347_rgbf000067-resize', '001268_2014-06-16_16-11-51_260595134347_rgbf000095-resize',
                  '001829_2014-06-26_21-40-41_260595134347_rgbf000107-resize', '002620_2014-06-24_14-43-00_094959634447_rgbf000065-resize',
                  '002537_2014-06-24_13-36-51_094959634447_rgbf000079-resize', '001603_2014-06-20_11-17-13_260595134347_rgbf000046-resize',
                  '001196_2014-06-17_16-01-27_260595134347_rgbf000069-resize', '002397_2014-06-28_20-20-43_260595134347_rgbf000027-resize',
                  '000077_2014-05-14_21-31-56_260595134347_rgbf000150-resize', '002940_2014-06-26_15-54-27_094959634447_rgbf000083-resize']

In [437]:
kinect2_subset = '/Users/prachi/Documents/depth_duration/kinect2data_subset'

for folder in exclude:
    path_remove = kinect2_subset + '/' + folder
    shutil.rmtree(path_remove)
    

In [439]:
import os

imagefolders = 0

for filenames in os.listdir(kinect2_subset):
    imagefolders += 1

print("Total image folders left: ", imagefolders)

Total image folders left:  721


In [9]:
import matplotlib.pyplot as plt
import numpy as np
import json
import matplotlib.image as mpimg
import random
from PIL import Image
import os
from pathlib import Path


def plotAll(folder, exit, width):
    """
    Creates folder for image with:
    - annotated image 
        - targets are only included if:
            (1) x = 364
            (2) target object is NOT floor or ceiling
            (3) y is inside the range defined as 2*width of target from image boundary (0, 529)
        - target y coordinate is the average of all y crossing points for that object 
    - original image 
    - data.json that includes data for ALL crossing points, not just those included in the annotated 
    """
#     data = json.load(open(jsonpath))
    data = allCenterObjects(folder, exit)
    img = mpimg.imread(data['imagePath'])
    plt.imshow(img)
    colors = randomColors(len(data['objects'])*4)

    for i in range(len(data['objects'])):
            
        crossing_points = data['objects'][i]['crossing_points']
        # all y coordinates that cross the midline
        ys = [crossing_points[i]['y'] for i in range(len(crossing_points))]
        depths = [crossing_points[i]['depth'] for i in range(len(crossing_points))]
        # include object name in the filename of the target image + dbID 
        # if the targets fall on any object in the list below, do not draw the target 
        obj_exclude = ['floor', 'ceiling']
        coord_object = data['objects'][i]['name'].lower()
        if coord_object not in obj_exclude:
        # target is placed at the AVERAGE of the y-coords so that the target is not on the edge of the object
        # JSON FILE ASSOCIATED WITH THIS TARGET NEEDS TO HAVE THE AVERAGE DEPTH ADDED 
        # JSON FILE needs to be exact same format as before 
            coord = np.mean(np.array(ys))
            target_depth = np.mean(np.array(depths))
            # exclude targets outside of this range [1,5] inclusive
            if 1.0 <= target_depth <= 5.0:
                # targets should not be within 2 target widths of image boundaries
                y_range = [2*width, 529 - (2*width)]
                if coord >= y_range[0] and coord <= y_range[1]:
                    plt.plot(364, coord, color=colors[i], marker='^', markersize=3, label=str(i) + ': ' + str(target_depth) + 'm')
                    # plt.annotate(i, (350, point['y']))
                    if i % 2 == 0:
                        xoff = 20
                    else:
                        xoff = -20
                    plt.annotate(i, xy=(364, coord), xytext=(xoff, -35),
                                     textcoords='offset points', ha='center', va='bottom', color='blue',
                                     bbox=dict(boxstyle='round,pad=0.2', fc='yellow', alpha=0.3),
                                     arrowprops=dict(arrowstyle='->', color='k'))
    plt.legend(loc='upper right')
#     unique = data['dbPath'][-57:-51] # this number is NOT unique 
    # folder name 
    unique = data['dbPath'].split("/")[-1]
    os.mkdir(exit + '/' + unique)
    plt.savefig(exit + '/' + unique + '/' + unique + '-annotated.png', dpi=500)
    plt.clf()
    with open(exit + '/' + unique + '/' + 'data.json', 'w') as json_file:
        json.dump(data, json_file)
    Image.open(data['imagePath']).copy().save(exit + '/' + unique + '/' + unique + '-original.jpg')


def randomColors(amount):
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(amount)]
    return color


def main_plotALL(source, destination, width):
    for _dir in os.listdir(source):
        path = source + '/' + _dir
        if '.DS_Store' not in path:
            try:
                plotAll(path, destination, width)
            except:
                print(path)



#### Run main_plotAll to create all the image folders

Ignore the Topological Errors - there are some abnormal coords in index.json file

In [11]:
kinectv2_subset = '/Users/prachi/Documents/depth_duration/kinect2data_subset'
exit = '/Users/prachi/Documents/depth_duration/mar3_depthDuration_stimuli/images_kinect2data_subset'
main_plotALL(kinectv2_subset, exit, 20)

# Topology Exception will happen for some images due to problems with Kinectv2 tagging --> NO cause for concern 

In [12]:
#targets.py EDITED by Prachi 
from PIL import Image, ImageDraw
import json
import os, json
import shutil 
from shutil import copy2


def drawTargets(path, alpha, width, dest):
    '''
    UPDATED:
    - takes mother image and loops through all acceptable targets
    - for each acceptable target creates a new folder for the target image in the destination folder 
    - this target folder include 
        - annotation img (all targets)
        - data.json for the specific target (created in this function)
        - original image 
        - target image 
    
    drawTargets(path, alpha, width, dest)
    alpha = opacity of the target out of 100 
    width = width of the target in pixels
    path = current location of the image 
    dest = location of image with the target drawn on 

    for example:
    drawTargets('/Users/prachi/Desktop/TEST/000002', 200, 20, test_exit)

    Targets are generated using data from the data.json file for each image.
    This file contains a list of objects whose outlines cross the centerline of the image, 
        as well as the y-coordinate and depth of each pixel where the centerline is crossed. 
    Most images have far more than 4 possible targets.

    Targets are placed at the average y location of all the coordinates that cross the midline

    '''
    tint = (0, 0, 0)
    num = path.split('/')[-1]

    im_path = path + '/' + num + "-" + "original.jpg"
    
    width = width / 2
    im = Image.open(im_path)
    im = im.convert('RGBA')
    tmp = Image.new('RGBA', im.size, tint + (0,))
    data = json.load(open(path + '/data.json'))
    
    new_json = dict()
    
    
    for i in range(len(data['objects'])):     
        crossing_points = data['objects'][i]['crossing_points']
        # all y coordinates that cross the midline
        ys = [crossing_points[i]['y'] for i in range(len(crossing_points))]
        depths = [crossing_points[i]['depth'] for i in range(len(crossing_points))]
        # include object name in the filename of the target image + dbID 
        # if the targets fall on any object in the list below, do not draw the target 
        obj_exclude = ['floor', 'ceiling']
        coord_object = data['objects'][i]['name'].lower()
        if coord_object not in obj_exclude:
        # target is placed at the AVERAGE of the y-coords so that the target is not on the edge of the object
            coord = np.mean(np.array(ys))
            target_depth = np.mean(np.array(depths))
            # exclude targets outside of this range [1,5] inclusive
            if 1.0 <= target_depth <= 5.0:
                # targets should not be within 2 target widths of image boundaries
                y_range = [2*width, 529 - (2*width)]
                if coord >= y_range[0] and coord <= y_range[1]:
                    lg = [(364 - width, coord - width), (364 + width, coord + width)]
                    med = [(366 - width, coord - width + 2), (362 + width, coord + width - 2)]
                    sm = [(368 - width, coord - width + 5), (360 + width, coord + width - 5)]
                    draw = ImageDraw.Draw(tmp)
                    draw.ellipse(lg, fill=(0, 0, 0, alpha))
                    # lime green: 50,205,50
                    draw.ellipse(med, fill=(50, 205, 50, alpha))
                #     draw.ellipse(med, fill=(255, 255, 255, alpha))
                    draw.ellipse(sm, fill=(255, 0, 0, alpha))
                    img = Image.alpha_composite(im, tmp)
                    # img.show()
                
                    
                    split_path = path.split("/")
                    folder_name = split_path[len(split_path)-1]
                    
                    #### Create new folder for target image ####
                    os.mkdir(str(dest + '/' + folder_name + '_' + str(i))) #creates new directory
                    
                    # Target Image 
                    img.save(dest + '/' + folder_name + '_' + str(i) + '/' + folder_name + '_' + str(i) + '-target.png')
                    tmp = Image.new('RGBA', im.size, tint + (0,))
                    
                    # Annotated Image Source path  
                    a_source = path + "/" + folder_name + "-" + "annotated.png"
                    # Annotated Image Destination path  
                    a_destination = dest + '/' + folder_name + '_' + str(i) 
                    # Copies the content of source to destination  
                    copy2(a_source, a_destination)
                    
                    # OG Image Source path  
                    og_source = path + "/" + folder_name + "-" + "original.jpg"
                    # OG Image Destination path  
                    og_destination = dest + '/' + folder_name + '_' + str(i)
                    # Copies the content of source to destination  
                    copy2(og_source, og_destination)
                    
                    # create JSON for the specific target image 
                    new_json["dbPath"] = data["dbPath"]
                    new_json["imagePath"] = data["imagePath"]
                    obj_dict = dict()
                    obj_dict["name"] = coord_object
                    obj_dict["dbID"] = data['objects'][i]["dbID"]
                    obj_dict["crossing_point"] = [{"x": 364,
                                                   "y": coord,
                                                   "depth": target_depth}]
                    new_json["objects"] = [obj_dict]
                    
                    # Serializing json  
                    json_object = json.dumps(new_json, indent = 4) 
                    # Writing to sample.json 
                    with open(dest + '/' + folder_name + '_' + str(i) + '/' + "data.json", "w") as outfile: 
                        outfile.write(json_object) 

                        
def main_drawTargets(rootdir, alpha, width, dest):
    for _dir in os.listdir(rootdir):
        path = rootdir + '/' + _dir
        if '.DS_Store' not in path:
            try:
                drawTargets(path, alpha, width, dest)
            except:
                print(path)

#### Run main_drawTargets to create all target folders

In [13]:
imagefolders = '/Users/prachi/Documents/depth_duration/mar3_depthDuration_stimuli/images_kinect2data_subset'

targetfolders = '/Users/prachi/Documents/depth_duration/mar3_depthDuration_stimuli/targetImages_kinect2data_subset'

main_drawTargets(imagefolders, 200, 20, targetfolders)


### Remove Target Images and Images Identified by Visual Inspection 

Criteria:

- poor object segmentation 
- irregular angle 
- extreme rotation on any dimension (x, y, z)
- confusing scene 
- poor depth tagging 
- ambiguous target placement

In [22]:
exclude_objSeg = ['000347_2014-06-09_19-49-03_260595134347_rgbf000225-resize', '003275_2014-05-14_16-10-31_094959634447_rgbf000100-resize',
                  '002120_2014-06-25_20-34-00_260595134347_rgbf000087-resize', '002486_2014-06-28_20-42-55_260595134347_rgbf000029-resize', 
                  '000354_2014-06-09_20-31-55_260595134347_rgbf000172-resize', '003351_2014-05-20_11-38-25_094959634447_rgbf000027-resize',
                  '002623_2014-06-24_14-44-20_094959634447_rgbf000038-resize', '001841_2014-06-22_13-18-35_260595134347_rgbf000048-resize',
                  '002475_2014-06-28_20-40-15_260595134347_rgbf000021-resize', '003474_2014-05-21_15-03-14_094959634447_rgbf002451-resize',
                  '003586_2014-05-23_16-47-01_094959634447_rgbf000100-resize', '002483_2014-06-28_20-42-10_260595134347_rgbf000029-resize',
                  '003154_2014-05-12_11-28-46_094959634447_rgbf000101-resize', '002808_2014-06-22_20-07-31_094959634447_rgbf000079-resize',
                  '002560_2014-06-24_13-48-11_094959634447_rgbf000075-resize', '000579_2014-06-08_11-07-06_260595134347_rgbf000166-resize']

exclude_other = ['002727_2014-06-22_11-49-24_094959634447_rgbf000025-resize', '002434_2014-06-28_20-30-03_260595134347_rgbf000054-resize',
                 '002093_2014-06-25_19-50-26_260595134347_rgbf000160-resize', 
                 '003044_2014-06-15_13-36-59_094959634447_rgbf000150-resize', '000903_2014-06-09_16-11-40_260595134347_rgbf000026-resize', 
                 '002091_2014-06-25_19-49-40_260595134347_rgbf000088-resize', 
                 '003025_2014-06-15_13-14-53_094959634447_rgbf000112-resize', '002291_2014-06-28_18-58-48_260595134347_rgbf000033-resize',
                 '003099_2014-06-24_21-04-53_094959634447_rgbf000076-resize', 
                 '002090_2014-06-25_19-49-03_260595134347_rgbf000064-resize', '001371_2014-06-19_15-53-51_260595134347_rgbf000049-resize',
                 '001931_2014-06-22_14-21-36_260595134347_rgbf000196-resize', '002443_2014-06-28_20-31-53_260595134347_rgbf000050-resize',
                 '000899_2014-06-09_15-49-21_260595134347_rgbf000086-resize', 
                 '000888_2014-06-09_15-43-40_260595134347_rgbf000119-resize',
                 '000689_2014-06-08_13-06-12_260595134347_rgbf000089-resize', '000877_2014-06-09_21-02-47_260595134347_rgbf000098-resize',
                 '002312_2014-06-28_19-58-29_260595134347_rgbf000084-resize', '000582_2014-06-09_16-27-17_260595134347_rgbf000224-resize',
                 '000954_2014-06-08_11-47-52_260595134347_rgbf000104-resize', '002142_2014-06-25_21-20-49_260595134347_rgbf000163-resize',
                 '002755_2014-06-22_19-10-02_094959634447_rgbf000076-resize', '001179_2014-06-17_15-47-33_260595134347_rgbf000093-resize',
                 '000924_2014-06-09_16-30-21_260595134347_rgbf000115-resize', 
                 '003510_2014-05-21_17-32-00_094959634447_rgbf000101-resize',
                 '002701_2014-06-22_11-27-34_094959634447_rgbf000075-resize',  
                 '002135_2014-06-25_21-17-43_260595134347_rgbf000087-resize', '000842_2014-06-08_17-05-51_260595134347_rgbf000104-resize',
                 '002382_2014-06-28_20-15-35_260595134347_rgbf000046-resize', '000442_2014-06-08_11-44-10_260595134347_rgbf000225-resize', 
                 '002133_2014-06-25_21-12-52_260595134347_rgbf000082-resize', '001067_2014-06-08_22-04-00_260595134347_rgbf000049-resize',
                 '003096_2014-06-24_21-03-34_094959634447_rgbf000070-resize', 
                 '002772_2014-06-22_19-22-49_094959634447_rgbf000059-resize']

exclude_other_round2 = ['003229_2014-05-14_13-43-11_094959634447_rgbf000100-resize', '001165_2014-06-17_15-36-50_260595134347_rgbf000127-resize',
                 '003532_2014-05-22_15-05-18_094959634447_rgbf000101-resize', '002172_2014-06-24_21-39-57_260595134347_rgbf000101-resize',
                 '003058_2014-06-15_14-02-44_094959634447_rgbf000150-resize', '001948_2014-06-24_15-42-56_260595134347_rgbf000060-resize',
                 
                 '000459_2014-06-08_16-54-11_260595134347_rgbf000096-resize', '002929_2014-06-26_15-49-31_094959634447_rgbf000076-resize',
                 '000621_2014-06-08_11-21-23_260595134347_rgbf000147-resize', '001820_2014-06-26_20-54-15_260595134347_rgbf000032-resize',
                 '002471_2014-06-28_20-39-28_260595134347_rgbf000025-resize',
                 '002396_2014-06-28_20-20-31_260595134347_rgbf000072-resize',
                 '000694_2014-06-08_13-12-30_260595134347_rgbf000219-resize', '002339_2014-06-28_20-05-27_260595134347_rgbf000042-resize',
                 '003557_2014-05-23_10-25-16_094959634447_rgbf006128-resize', '000566_2014-06-09_22-50-50_260595134347_rgbf000080-resize',
                 '002402_2014-06-28_20-21-52_260595134347_rgbf000023-resize', '002794_2014-06-22_19-38-01_094959634447_rgbf000082-resize',
                 '002092_2014-06-25_19-49-59_260595134347_rgbf000128-resize', '002430_2014-06-28_20-29-09_260595134347_rgbf000047-resize',
                 '002547_2014-06-24_13-39-39_094959634447_rgbf000079-resize', '002651_2014-06-24_14-57-10_094959634447_rgbf000064-resize',
                 '002354_2014-06-28_20-08-27_260595134347_rgbf000049-resize', '003136_2014-05-12_10-58-05_094959634447_rgbf000100-resize']

exclude_round3 = ['003187_2014-05-12_22-08-19_094959634447_rgbf000051-resize', '003041_2014-06-15_13-35-37_094959634447_rgbf000077-resize',
                  '002381_2014-06-28_20-15-26_260595134347_rgbf000027-resize', '001293_2014-06-16_16-27-24_260595134347_rgbf000103-resize',
                  '002724_2014-06-22_11-47-39_094959634447_rgbf000080-resize', '002520_2014-06-24_13-24-20_094959634447_rgbf000076-resize',
                  '001613_2014-06-20_11-32-48_260595134347_rgbf000054-resize', '001182_2014-06-17_15-51-16_260595134347_rgbf000070-resize',
                  '002601_2014-06-24_14-28-15_094959634447_rgbf000032-resize', '002436_2014-06-28_20-30-21_260595134347_rgbf000094-resize',
                  '003383_2014-05-20_14-41-21_094959634447_rgbf000101-resize', '002552_2014-06-24_13-46-11_094959634447_rgbf000074-resize',
                  '000790_2014-06-08_22-35-19_260595134347_rgbf000085-resize', '003409_2014-05-21_09-12-10_094959634447_rgbf000100-resize',
                  '002176_2014-06-25_19-50-50_260595134347_rgbf000073-resize', '000348_2014-06-09_19-49-30_260595134347_rgbf000130-resize',
                  '002331_2014-06-28_20-03-43_260595134347_rgbf000074-resize', '002834_2014-06-22_20-37-29_094959634447_rgbf000089-resize',
                  '003639_2014-05-23_22-09-41_094959634447_rgbf000055-resize', '002470_2014-06-28_20-39-13_260595134347_rgbf000029-resize',
                  '000487_2014-06-09_20-47-43_260595134347_rgbf000173-resize', '003227_2014-05-14_13-42-10_094959634447_rgbf000100-resize',
                  '002536_2014-06-24_13-36-30_094959634447_rgbf000088-resize', '003591_2014-05-23_16-57-22_094959634447_rgbf002866-resize',
                  '000002_2014-05-26_14-23-37_260595134347_rgbf000103-resize', '001513_2014-06-20_16-23-55_260595134347_rgbf000118-resize',
                  '001339_2014-06-16_17-04-45_260595134347_rgbf000065-resize', '003295_2014-05-14_22-20-22_094959634447_rgbf000100-resize',
                  '001430_2014-06-19_16-50-36_260595134347_rgbf000047-resize', '000370_2014-06-09_15-58-58_260595134347_rgbf000076-resize',
                  '001427_2014-06-19_16-49-12_260595134347_rgbf000035-resize', '002961_2014-06-08_17-40-53_094959634447_rgbf000144-resize',
                  '001435_2014-06-19_16-52-03_260595134347_rgbf000067-resize', '001268_2014-06-16_16-11-51_260595134347_rgbf000095-resize',
                  '001829_2014-06-26_21-40-41_260595134347_rgbf000107-resize', '002620_2014-06-24_14-43-00_094959634447_rgbf000065-resize',
                  '002537_2014-06-24_13-36-51_094959634447_rgbf000079-resize', '001603_2014-06-20_11-17-13_260595134347_rgbf000046-resize',
                  '001196_2014-06-17_16-01-27_260595134347_rgbf000069-resize', '002397_2014-06-28_20-20-43_260595134347_rgbf000027-resize',
                  '000077_2014-05-14_21-31-56_260595134347_rgbf000150-resize', '002940_2014-06-26_15-54-27_094959634447_rgbf000083-resize',
                  '000811_2014-06-08_17-20-18_260595134347_rgbf000128-resize', '001286_2014-06-16_16-24-19_260595134347_rgbf000118-resize',
                  '002095_2014-06-25_19-51-27_260595134347_rgbf000113-resize']

In [416]:
target_images = '/Users/prachi/Documents/depth_duration/mar3_depthDuration_stimuli/targetImages_kinect2data_subset'
num_imgs_excluded = 0

for targetfolder in os.listdir(target_images):
    if '.DS_Store' not in targetfolder:
        if len(targetfolder) == 59:
            img = targetfolder[:-2]
            if img in exclude_objSeg:
                num_imgs_excluded += 1
                shutil.rmtree(target_images + '/' + targetfolder)
        if len(targetfolder) == 60:
            img = targetfolder[:-3]
            if img in exclude_objSeg:
                num_imgs_excluded += 1
                shutil.rmtree(target_images + '/' + targetfolder)

num_imgs_excluded

76

In [417]:
target_images = '/Users/prachi/Documents/depth_duration/mar3_depthDuration_stimuli/targetImages_kinect2data_subset'
num_imgs_excluded = 0

for targetfolder in os.listdir(target_images):
    if '.DS_Store' not in targetfolder:
        if len(targetfolder) == 59:
            img = targetfolder[:-2]
            if img in exclude_rotation:
                num_imgs_excluded += 1
                shutil.rmtree(target_images + '/' + targetfolder)
        if len(targetfolder) == 60:
            img = targetfolder[:-3]
            if img in exclude_rotation:
                num_imgs_excluded += 1
                shutil.rmtree(target_images + '/' + targetfolder)

num_imgs_excluded

63

In [418]:
target_images = '/Users/prachi/Documents/depth_duration/mar3_depthDuration_stimuli/targetImages_kinect2data_subset'
num_imgs_excluded = 0

for targetfolder in os.listdir(target_images):
    if '.DS_Store' not in targetfolder:
        if len(targetfolder) == 59:
            img = targetfolder[:-2]
            if img in exclude_other:
                num_imgs_excluded += 1
                shutil.rmtree(target_images + '/' + targetfolder)
        if len(targetfolder) == 60:
            img = targetfolder[:-3]
            if img in exclude_other:
                num_imgs_excluded += 1
                shutil.rmtree(target_images + '/' + targetfolder)

num_imgs_excluded

136

In [419]:
target_images = '/Users/prachi/Documents/depth_duration/mar3_depthDuration_stimuli/targetImages_kinect2data_subset'
num_imgs_excluded = 0

for targetfolder in os.listdir(target_images):
    if '.DS_Store' not in targetfolder:
        if len(targetfolder) == 59:
            img = targetfolder[:-2]
            if img in exclude_other_round2:
                num_imgs_excluded += 1
                shutil.rmtree(target_images + '/' + targetfolder)
        if len(targetfolder) == 60:
            img = targetfolder[:-3]
            if img in exclude_other_round2:
                num_imgs_excluded += 1
                shutil.rmtree(target_images + '/' + targetfolder)

num_imgs_excluded

89

In [23]:
target_images = '/Users/prachi/Documents/depth_duration/mar3_depthDuration_stimuli/targetImages_kinect2data_subset'
num_imgs_excluded = 0

for targetfolder in os.listdir(target_images):
    if '.DS_Store' not in targetfolder:
        if len(targetfolder) == 59:
            img = targetfolder[:-2]
            if img in exclude_round3:
                num_imgs_excluded += 1
                shutil.rmtree(target_images + '/' + targetfolder)
        if len(targetfolder) == 60:
            img = targetfolder[:-3]
            if img in exclude_round3:
                num_imgs_excluded += 1
                shutil.rmtree(target_images + '/' + targetfolder)

num_imgs_excluded

6

#### Remove target images identified after visual inspection (poor depth tagging or ambiguous placement)


In [24]:
bad_targets = ['002467_2014-06-28_20-38-07_260595134347_rgbf000026-resize_3',
               '000757_2014-06-08_22-03-00_260595134347_rgbf000104-resize_1',
               '000483_2014-06-09_20-41-45_260595134347_rgbf000116-resize_3',
               '001603_2014-06-20_11-17-13_260595134347_rgbf000046-resize_1',
               '001603_2014-06-20_11-17-13_260595134347_rgbf000046-resize_2',
               '000940_2014-06-09_17-02-13_260595134347_rgbf000182-resize_4',
               '000496_2014-06-08_22-50-40_260595134347_rgbf000056-resize_6',
               '000496_2014-06-08_22-50-40_260595134347_rgbf000056-resize_4',
               '003488_2014-05-21_15-49-28_094959634447_rgbf000100-resize_1',
               '002326_2014-06-28_20-02-31_260595134347_rgbf000017-resize_1',
               '002326_2014-06-28_20-02-31_260595134347_rgbf000017-resize_2',
               '000103_2014-05-14_22-01-17_260595134347_rgbf000152-resize_2',
               '000103_2014-05-14_22-01-17_260595134347_rgbf000152-resize_3',
               '000727_2014-06-08_17-30-41_260595134347_rgbf000132-resize_3',
               '000727_2014-06-08_17-30-41_260595134347_rgbf000132-resize_0',
               '000725_2014-06-08_17-29-20_260595134347_rgbf000120-resize_3',
               '002558_2014-06-24_13-47-41_094959634447_rgbf000087-resize_0',
               '001425_2014-06-19_16-45-40_260595134347_rgbf000051-resize_3',
               '001167_2014-06-17_15-38-07_260595134347_rgbf000108-resize_4',
               '002209_2014-06-28_18-36-10_260595134347_rgbf000051-resize_5',
               '002209_2014-06-28_18-36-10_260595134347_rgbf000051-resize_4',
               '001536_2014-06-20_16-52-44_260595134347_rgbf000092-resize_5',
               '002069_2014-06-24_21-45-10_260595134347_rgbf000071-resize_1',
               '001427_2014-06-19_16-49-12_260595134347_rgbf000035-resize_3',
               '001427_2014-06-19_16-49-12_260595134347_rgbf000035-resize_2',
               '001436_2014-06-19_16-52-40_260595134347_rgbf000062-resize_3',
               '001559_2014-06-20_17-13-59_260595134347_rgbf000081-resize_0',
               '000743_2014-06-08_18-48-36_260595134347_rgbf000087-resize_5',
               '001235_2014-06-17_16-37-36_260595134347_rgbf000084-resize_5',
               '001432_2014-06-19_16-51-16_260595134347_rgbf000077-resize_4',
               '001033_2014-06-08_13-23-43_260595134347_rgbf000055-resize_0',
               '000526_2014-06-08_13-56-05_260595134347_rgbf000128-resize_4',
               '001570_2014-06-20_17-21-09_260595134347_rgbf000135-resize_2',
               '002545_2014-06-24_13-38-53_094959634447_rgbf000080-resize_3',
               '000918_2014-06-09_22-45-57_260595134347_rgbf000166-resize_4',
               '000740_2014-06-08_18-46-23_260595134347_rgbf000049-resize_4',
               '003125_2014-05-12_10-39-22_094959634447_rgbf000100-resize_1',
               '002702_2014-06-22_11-27-56_094959634447_rgbf000141-resize_2',
               '001092_2014-06-15_17-34-58_260595134347_rgbf000171-resize_0',
               '000936_2014-06-09_16-55-37_260595134347_rgbf000153-resize_4',
               '001815_2014-06-26_20-52-55_260595134347_rgbf000020-resize_5',
               '001779_2014-06-26_20-01-23_260595134347_rgbf000040-resize_4',
               '001016_2014-06-09_16-16-10_260595134347_rgbf000112-resize_3',
               '002831_2014-06-22_20-36-26_094959634447_rgbf000082-resize_3',
               '000627_2014-06-08_16-23-54_260595134347_rgbf000173-resize_3',
               '001219_2014-06-17_16-20-25_260595134347_rgbf000111-resize_1',
               '000481_2014-06-09_20-40-39_260595134347_rgbf000199-resize_0',
               '000633_2014-06-08_16-33-28_260595134347_rgbf000107-resize_2',
               '002464_2014-06-28_20-36-57_260595134347_rgbf000029-resize_3',
               '001181_2014-06-17_15-50-57_260595134347_rgbf000106-resize_2',
               '000368_2014-06-09_15-53-01_260595134347_rgbf000155-resize_0',
               '003149_2014-05-12_11-17-52_094959634447_rgbf003779-resize_6',
               '000569_2014-06-09_22-51-47_260595134347_rgbf000141-resize_1',
               '000884_2014-06-09_15-41-53_260595134347_rgbf000112-resize_1',
               '000761_2014-06-08_22-06-20_260595134347_rgbf000124-resize_2',
               '001774_2014-06-26_20-00-11_260595134347_rgbf000023-resize_0', 
               '001774_2014-06-26_20-00-11_260595134347_rgbf000023-resize_5',
               '000677_2014-06-08_23-10-32_260595134347_rgbf000158-resize_2',
               '001502_2014-06-19_17-51-09_260595134347_rgbf000104-resize_3',
               '001536_2014-06-20_16-52-44_260595134347_rgbf000092-resize_1']

In [26]:
target_images = '/Users/prachi/Documents/depth_duration/mar3_depthDuration_stimuli/targetImages_kinect2data_subset'

count_removed = 0
for targ_folder in bad_targets:
    p = target_images + '/' + targ_folder 
    count_removed += 1
    try:
        shutil.rmtree(p)
    except:
        print("Already removed")
    
print(count_removed)

In [ ]:

count_imgs_excluded = 0
for targetfolder in os.listdir(target_images):
    if '.DS_Store' not in targetfolder:
        image = targetfolder.split('_')[0]
        if image in images_exclude:
            count_imgs_excluded += 1
            path = target_images + '/' + targetfolder
            shutil.rmtree(path)
count_imgs_excluded

In [ ]:
# since images have already been removed, count of targets removed is fewer than the length of the target exclude list

count_targets_excluded = 0
for targetfolder in exclude_objSeg:
    if len(targetfolder) == 59:
        imgfolder = targetfolder[:-2]
        
    if len(targetfolder) == 60:
        imgfolder = targetfolder[:-3]
    count_targets_excluded += 1
    path = target_images + '/' + targetfolder
#     shutil.rmtree(path)
    
            
count_targets_excluded

### Misc. Helper Functions

In [12]:
#cleaning.py
from pathlib import Path
import json
from PIL import Image
import numpy as np
import math
import os


def getAllDirs(path):
    dir = Path(path)
    dirs = []
    for sub in dir.iterdir():
        dirs.append(str(sub))
    return dirs


def jsonFails(path):
    fails = 0
    dirs = getAllDirs(path)
    faildirs = []
    for sub in dirs:
        try:
            with open(str(sub) + '/annotation2Dfinal/index.json') as json_file:
                json.load(json_file)
        except Exception:
            fails = fails + 1
            faildirs.append(str(sub))
    print(str(fails))
    return faildirs


def makeDepthArray(depthpath):
    img = Image.open(depthpath)
    img.load()
    data = np.asarray(img)
    data = np.dot(1/8000.0, data)
    return data


def averageDepth(path):
    subs = getAllDirs(path)
    depths = []
    for sub in subs:
        try:
            data = json.load(open(str(sub) + '/data.json'))
        except Exception:
            continue
        split = data['imagePath'].split('image')
        depthpath = split[0] + 'depth_bfx' + split[1][:-3] + 'png'
        depths.append(makeDepthArray(depthpath))
    # return np.mean(np.asarray(depths), axis=0)
    return np.asarray(depths)


def allTargets(path):
    subs = getAllDirs(path)
    objects = []
    for sub in subs:
        try:
            data = json.load(open(str(sub) + '/data.json'))
        except Exception:
            continue
        for obj in data['objects']:
            for point in obj['crossing_points']:
                point['name'] = obj['name']
                objects.append(point)
    return objects


def allYCoords(path):
    subs = getAllDirs(path)
    coords = []
    for sub in subs:
        try:
            data = json.load(open(str(sub)))
        except Exception:
            continue
        for obj in data['objects']:
            for point in obj['crossing_points']:
                coords.append(point['y'])
    return coords


def yaw(matrixtext):
    x21 = float(matrixtext.split('\n')[1].split(' ')[0])
    x11 = float(matrixtext.split('\n')[0].split(' ')[0])
    return math.degrees(np.arctan(x21 / x11))


def filterByTilt(path, tilt):
    dirs = getAllDirs(path)
    untilted = []
    for sub in dirs:
        try:
            data = json.load(open(str(sub)))
        except Exception:
            continue
        extpath = data['dbPath'] + '/extrinsics'
        fname = os.listdir(extpath)[0]
        text = open(extpath + '/' + fname).read()
        if abs(yaw(text)) <= tilt:
            untilted.append(str(sub))
    return untilted


def byCount(path, objects):
    dirs = getAllDirs(path)
    enough = []
    for sub in dirs:
        try:
            data = json.load(open(str(sub) + '/data.json'))
        except Exception:
            continue
        if len(data['objects']) < objects:
            enough.append(str(sub))
    return(np.asarray(enough))
